# GPT-4o MiniとRAGを組み合わせる方法 - 服装マッチメーカーアプリの作成

服装マッチメーカーアプリのJupyter Notebookへようこそ！このプロジェクトは、衣類アイテムの画像を分析し、色、スタイル、タイプなどの主要な特徴を抽出するGPT-4o miniモデルの力を実証します。私たちのアプリの核心は、OpenAIによって開発されたこの先進的な画像分析モデルに依存しており、入力された衣類アイテムの特性を正確に識別することを可能にします。

GPT-4o miniは、自然言語処理と画像認識を組み合わせた小型モデルで、低レイテンシでテキストと視覚的入力の両方に基づいて理解し、応答を生成することができます。

GPT-4o miniモデルの機能を基盤として、カスタムマッチングアルゴリズムとRAG技術を採用し、識別された特徴を補完するアイテムを知識ベースから検索します。このアルゴリズムは、色の相性やスタイルの一貫性などの要因を考慮して、ユーザーに適切な推奨事項を提供します。このノートブックを通じて、衣類推奨システムの作成におけるこれらの技術の実用的な応用を紹介することを目指しています。

GPT-4o mini + RAG（Retrieval-Augmented Generation）の組み合わせを使用することで、いくつかの利点が得られます：

1. **文脈的理解**: GPT-4o miniは入力画像を分析し、描かれているオブジェクト、シーン、活動などの文脈を理解できます。これにより、インテリアデザイン、料理、教育など、様々な分野でより正確で関連性の高い提案や情報を提供できます。

2. **豊富な知識ベース**: RAGは、GPT-4の生成機能と、様々な分野にわたる大規模な情報コーパスにアクセスする検索コンポーネントを組み合わせます。これは、歴史的事実から科学的概念まで、幅広い知識に基づいて提案や洞察を提供できることを意味します。

3. **カスタマイゼーション**: このアプローチにより、様々なアプリケーションにおいて特定のユーザーニーズや好みに対応するための簡単なカスタマイゼーションが可能になります。ユーザーの芸術的嗜好に合わせた提案の調整や、学生の学習レベルに基づいた教育コンテンツの提供など、システムはパーソナライズされた体験を提供するように適応できます。

全体的に、GPT-4o mini + RAGアプローチは、生成型と検索ベースのAI技術の両方の強みを活用し、様々なファッション関連アプリケーションに対して高速で強力、かつ柔軟なソリューションを提供します。

### 環境設定

まず必要な依存関係をインストールし、その後ライブラリをインポートして、後で使用するユーティリティ関数を記述します。

In [ ]:
%pip install openai --quiet
%pip install tenacity --quiet
%pip install tqdm --quiet
%pip install numpy --quiet
%pip install typing --quiet
%pip install tiktoken --quiet
%pip install concurrent --quiet

In [ ]:
import pandas as pd
import numpy as np
import json
import ast
import tiktoken
import concurrent
from openai import OpenAI
from tqdm import tqdm
from tenacity import retry, wait_random_exponential, stop_after_attempt
from IPython.display import Image, display, HTML
from typing import List

client = OpenAI()

GPT_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-large"
EMBEDDING_COST_PER_1K_TOKENS = 0.00013

### エンベディングの作成
データベースを選択し、そのためのエンベディングを生成することで、ナレッジベースを設定します。この例では、dataフォルダにある`sample_styles.csv`ファイルを使用しています。これは約44,000アイテムを含むより大きなデータセットのサンプルです。このステップは、既製のベクトルデータベースを使用することで置き換えることも可能です。例えば、ベクトルデータベースを設定するために[これらのクックブック](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases)のいずれかに従うことができます。

In [ ]:
styles_filepath = "data/sample_clothes/sample_styles.csv"
styles_df = pd.read_csv(styles_filepath, on_bad_lines='skip')
print(styles_df.head())
print("Opened dataset successfully. Dataset has {} items of clothing.".format(len(styles_df)))

次に、データセット全体の埋め込みを生成します。これらの埋め込みの実行を並列化することで、より大きなデータセットに対してもスクリプトがスケールアップできるようにします。この手法により、`44K`エントリの完全なデータセットの埋め込み作成時間は、約4時間から約2-3分に短縮されます。

In [ ]:
## Batch Embedding Logic

# Simple function to take in a list of text objects and return them as a list of embeddings
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(10))
def get_embeddings(input: List):
    response = client.embeddings.create(
        input=input,
        model=EMBEDDING_MODEL
    ).data
    return [data.embedding for data in response]


# Splits an iterable into batches of size n.
def batchify(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx : min(ndx + n, l)]
     

# Function for batching and parallel processing the embeddings
def embed_corpus(
    corpus: List[str],
    batch_size=64,
    num_workers=8,
    max_context_len=8191,
):
    # Encode the corpus, truncating to max_context_len
    encoding = tiktoken.get_encoding("cl100k_base")
    encoded_corpus = [
        encoded_article[:max_context_len] for encoded_article in encoding.encode_batch(corpus)
    ]

    # Calculate corpus statistics: the number of inputs, the total number of tokens, and the estimated cost to embed
    num_tokens = sum(len(article) for article in encoded_corpus)
    cost_to_embed_tokens = num_tokens / 1000 * EMBEDDING_COST_PER_1K_TOKENS
    print(
        f"num_articles={len(encoded_corpus)}, num_tokens={num_tokens}, est_embedding_cost={cost_to_embed_tokens:.2f} USD"
    )

    # Embed the corpus
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        
        futures = [
            executor.submit(get_embeddings, text_batch)
            for text_batch in batchify(encoded_corpus, batch_size)
        ]

        with tqdm(total=len(encoded_corpus)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(batch_size)

        embeddings = []
        for future in futures:
            data = future.result()
            embeddings.extend(data)

        return embeddings
    

# Function to generate embeddings for a given column in a DataFrame
def generate_embeddings(df, column_name):
    # Initialize an empty list to store embeddings
    descriptions = df[column_name].astype(str).tolist()
    embeddings = embed_corpus(descriptions)

    # Add the embeddings as a new column to the DataFrame
    df['embeddings'] = embeddings
    print("Embeddings created successfully.")

#### 埋め込みを作成するための2つのオプション：
次の行では、サンプルの衣類データセットの**埋め込みを作成**します。処理に約0.02秒、結果をローカルの.csvファイルに書き込むのに約30秒かかります。このプロセスでは、`$0.00013/1K`トークンで価格設定されている`text_embedding_3_large`モデルを使用しています。データセットには約`1K`のエントリがあるため、以下の操作には約`$0.001`のコストがかかります。`44K`エントリの完全なデータセットで作業することを決定した場合、この操作には処理に2-3分かかり、約`$0.07`のコストがかかります。

**独自の埋め込みを作成したくない場合**、事前計算された埋め込みのデータセットを使用します。このセルをスキップして、次のセルのコードのコメントアウトを解除し、事前計算されたベクトルの読み込みを進めることができます。この操作では、すべてのデータをメモリに読み込むのに約1分かかります。

In [ ]:
generate_embeddings(styles_df, 'productDisplayName')
print("Writing embeddings to file ...")
styles_df.to_csv('data/sample_clothes/sample_styles_with_embeddings.csv', index=False)
print("Embeddings successfully stored in sample_styles_with_embeddings.csv")

In [ ]:
# styles_df = pd.read_csv('data/sample_clothes/sample_styles_with_embeddings.csv', on_bad_lines='skip')

# # Convert the 'embeddings' column from string representations of lists to actual lists of floats
# styles_df['embeddings'] = styles_df['embeddings'].apply(lambda x: ast.literal_eval(x))

print(styles_df.head())
print("Opened dataset successfully. Dataset has {} items of clothing along with their embeddings.".format(len(styles_df)))

### マッチングアルゴリズムの構築

このセクションでは、データフレーム内の類似アイテムを見つけるためのコサイン類似度検索アルゴリズムを開発します。この目的のために、カスタムのコサイン類似度関数を使用します。`sklearn`ライブラリには組み込みのコサイン類似度関数が提供されていますが、SDKの最近のアップデートにより互換性の問題が発生したため、独自の標準的なコサイン類似度計算を実装することにしました。

ベクトルデータベースがすでに設定されている場合は、この手順をスキップできます。ほとんどの標準的なデータベースには独自の検索機能が備わっており、このガイドで説明する後続の手順が簡素化されます。ただし、マッチングアルゴリズムは特定の閾値や返されるマッチ数の指定など、特定の要件に合わせてカスタマイズできることを実証することを目的としています。

`find_similar_items`関数は4つのパラメータを受け取ります：
- `embedding`: マッチを見つけたい対象の埋め込み
- `embeddings`: 最適なマッチを検索する埋め込みのリスト
- `threshold`（オプション）: このパラメータは、マッチが有効と見なされるための最小類似度スコアを指定します。閾値が高いほど、より近い（より良い）マッチが得られ、閾値が低いほど、より多くのアイテムが返されますが、初期の`embedding`との一致度は低くなる可能性があります
- `top_k`（オプション）: このパラメータは、指定された閾値を超えるアイテムの返却数を決定します。これらは、提供された`embedding`に対する最高スコアのマッチとなります

In [ ]:
def cosine_similarity_manual(vec1, vec2):
    """Calculate the cosine similarity between two vectors."""
    vec1 = np.array(vec1, dtype=float)
    vec2 = np.array(vec2, dtype=float)


    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)


def find_similar_items(input_embedding, embeddings, threshold=0.5, top_k=2):
    """Find the most similar items based on cosine similarity."""
    
    # Calculate cosine similarity between the input embedding and all other embeddings
    similarities = [(index, cosine_similarity_manual(input_embedding, vec)) for index, vec in enumerate(embeddings)]
    
    # Filter out any similarities below the threshold
    filtered_similarities = [(index, sim) for index, sim in similarities if sim >= threshold]
    
    # Sort the filtered similarities by similarity score
    sorted_indices = sorted(filtered_similarities, key=lambda x: x[1], reverse=True)[:top_k]

    # Return the top-k most similar items
    return sorted_indices

In [ ]:
def find_matching_items_with_rag(df_items, item_descs):
   """Take the input item descriptions and find the most similar items based on cosine similarity for each description."""
   
   # Select the embeddings from the DataFrame.
   embeddings = df_items['embeddings'].tolist()

   
   similar_items = []
   for desc in item_descs:
      
      # Generate the embedding for the input item
      input_embedding = get_embeddings([desc])
    
      # Find the most similar items based on cosine similarity
      similar_indices = find_similar_items(input_embedding, embeddings, threshold=0.6)
      similar_items += [df_items.iloc[i] for i in similar_indices]
    
   return similar_items

### 分析モジュール

このモジュールでは、`gpt-4o-mini`を活用して入力画像を分析し、詳細な説明、スタイル、タイプなどの重要な特徴を抽出します。分析は単純なAPI呼び出しを通じて実行され、分析対象の画像のURLを提供し、モデルに関連する特徴を識別するよう要求します。

モデルが正確な結果を返すことを確保するため、プロンプトで特定の技術を使用しています：

1. **出力フォーマットの指定**: モデルに対して、事前定義された構造を持つJSONブロックを返すよう指示します。構造は以下で構成されます：
   - `items` (str[]): 文字列のリスト。各文字列は、スタイル、色、性別を含む衣類アイテムの簡潔なタイトルを表します。これらのタイトルは、元のデータベースの`productDisplayName`プロパティに密接に似ています。
   - `category` (str): 与えられたアイテムを最もよく表すカテゴリ。モデルは、元のスタイルデータフレームに存在するすべての一意の`articleTypes`のリストから選択します。
   - `gender` (str): アイテムが対象とする性別を示すラベル。モデルは`[Men, Women, Boys, Girls, Unisex]`のオプションから選択します。

2. **明確で簡潔な指示**: 
   - アイテムタイトルに含めるべき内容と出力フォーマットについて明確な指示を提供します。出力はJSON形式である必要がありますが、モデルレスポンスに通常含まれる`json`タグは除きます。

3. **ワンショット例**: 
   - 期待される出力をさらに明確にするため、モデルに入力説明の例と対応する出力例を提供します。これによりトークン数が増加し（したがって呼び出しコストも増加）する可能性がありますが、モデルをガイドし、全体的なパフォーマンスの向上につながります。

この構造化されたアプローチに従うことで、`gpt-4o-mini`モデルから正確で有用な情報を取得し、さらなる分析とデータベースへの統合を目指しています。

In [ ]:
def analyze_image(image_base64, subcategories):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": f"""Given an image of an item of clothing, analyze the item and generate a JSON output with the following fields: "items", "category", and "gender".
                           Use your understanding of fashion trends, styles, and gender preferences to provide accurate and relevant suggestions for how to complete the outfit.
                           The items field should be a list of items that would go well with the item in the picture. Each item should represent a title of an item of clothing that contains the style, color, and gender of the item.
                           The category needs to be chosen between the types in this list: {subcategories}.
                           You have to choose between the genders in this list: [Men, Women, Boys, Girls, Unisex]
                           Do not include the description of the item in the picture. Do not include the ```json ``` tag in the output.

                           Example Input: An image representing a black leather jacket.

                           Example Output: {{"items": ["Fitted White Women's T-shirt", "White Canvas Sneakers", "Women's Black Skinny Jeans"], "category": "Jackets", "gender": "Women"}}
                           """,
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{image_base64}",
                },
                }
            ],
            }
        ]
    )
    # Extract relevant features from the response
    features = response.choices[0].message.content
    return features

### サンプル画像を使用したプロンプトのテスト

プロンプトの効果を評価するために、データセットから選択した画像を読み込んでテストしてみましょう。`"data/sample_clothes/sample_images"`フォルダから、様々なスタイル、性別、タイプの画像を使用します。選択したサンプルは以下の通りです：

- `2133.jpg`: メンズシャツ
- `7143.jpg`: レディースシャツ
- `4226.jpg`: カジュアルなメンズプリントTシャツ

これらの多様な画像でプロンプトをテストすることで、異なるタイプの衣類やアクセサリーから関連する特徴を正確に分析・抽出する能力を評価することができます。

.jpg画像をbase64でエンコードするためのユーティリティ関数が必要です。

In [ ]:
import base64

def encode_image_to_base64(image_path):
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read())
        return encoded_image.decode('utf-8')

In [ ]:
# Set the path to the images and select a test image
image_path = "data/sample_clothes/sample_images/"
test_images = ["2133.jpg", "7143.jpg", "4226.jpg"]

# Encode the test image to base64
reference_image = image_path + test_images[0]
encoded_image = encode_image_to_base64(reference_image)

In [ ]:
# Select the unique subcategories from the DataFrame
unique_subcategories = styles_df['articleType'].unique()

# Analyze the image and return the results
analysis = analyze_image(encoded_image, unique_subcategories)
image_analysis = json.loads(analysis)

# Display the image and the analysis results
display(Image(filename=reference_image))
print(image_analysis)

次に、画像解析の出力を処理し、それを使用してデータセットから一致するアイテムをフィルタリングして表示します。以下はコードの詳細です：

1. **画像解析結果の抽出**: `image_analysis`辞書からアイテムの説明、カテゴリ、性別を抽出します。

2. **データセットのフィルタリング**: `styles_df`DataFrameをフィルタリングして、画像解析から得られた性別と一致する（またはユニセックスの）アイテムのみを含め、解析された画像と同じカテゴリのアイテムは除外します。

3. **一致するアイテムの検索**: `find_matching_items_with_rag`関数を使用して、解析された画像から抽出された説明と一致するアイテムをフィルタリングされたデータセット内で検索します。

4. **一致するアイテムの表示**: 一致するアイテムの画像を表示するためのHTML文字列を作成します。アイテムIDを使用して画像パスを構築し、各画像をHTML文字列に追加します。最後に、`display(HTML(html))`を使用してノートブック内で画像をレンダリングします。

このセルは、画像解析の結果を使用してデータセットをフィルタリングし、解析された画像の特徴と一致するアイテムを視覚的に表示する方法を効果的に実演しています。

In [ ]:
# Extract the relevant features from the analysis
item_descs = image_analysis['items']
item_category = image_analysis['category']
item_gender = image_analysis['gender']


# Filter data such that we only look through the items of the same gender (or unisex) and different category
filtered_items = styles_df.loc[styles_df['gender'].isin([item_gender, 'Unisex'])]
filtered_items = filtered_items[filtered_items['articleType'] != item_category]
print(str(len(filtered_items)) + " Remaining Items")

# Find the most similar items based on the input item descriptions
matching_items = find_matching_items_with_rag(filtered_items, item_descs)

# Display the matching items (this will display 2 items for each description in the image analysis)
html = ""
paths = []
for i, item in enumerate(matching_items):
    item_id = item['id']
        
    # Path to the image file
    image_path = f'data/sample_clothes/sample_images/{item_id}.jpg'
    paths.append(image_path)
    html += f'<img src="{image_path}" style="display:inline;margin:1px"/>'

# Print the matching item description as a reminder of what we are looking for
print(item_descs)
# Display the image
display(HTML(html))

### ガードレール

GPT-4o miniのような大規模言語モデル（LLM）を使用する文脈において、「ガードレール」とは、モデルの出力が望ましいパラメータや境界内に留まることを確保するために設置されるメカニズムや検査を指します。これらのガードレールは、特に複雑で微妙なタスクを扱う際に、モデルの応答の品質と関連性を維持するために重要です。

ガードレールは以下のような理由で有用です：

1. **精度**: 提供された入力に対してモデルの出力が正確で関連性があることを確保します。
2. **一貫性**: 類似または関連する入力を扱う際に、モデルの応答の一貫性を維持します。
3. **安全性**: モデルが有害、攻撃的、または不適切なコンテンツを生成することを防ぎます。
4. **文脈的関連性**: モデルの出力が、使用される特定のタスクやドメインに文脈的に関連していることを確保します。

私たちの場合、GPT-4o miniを使用してファッション画像を分析し、元の衣装を補完するアイテムを提案しています。ガードレールを実装するために、**結果を精緻化**することができます：GPT-4o miniから初期提案を得た後、元の画像と提案されたアイテムをモデルに送り返すことができます。そして、提案された各アイテムが実際に元の衣装に適しているかどうかをGPT-4o miniに評価してもらうことができます。

これにより、モデルはフィードバックや追加情報に基づいて自己修正し、自身の出力を調整する能力を得ます。これらのガードレールを実装し、自己修正を可能にすることで、ファッション分析と推奨の文脈におけるモデル出力の信頼性と有用性を向上させることができます。

これを促進するために、提案されたアイテムが元の衣装にマッチするかどうかという質問に対して、LLMに単純な「はい」または「いいえ」の回答を求めるプロンプトを作成します。この二進的な応答は、精緻化プロセスを合理化し、モデルから明確で実行可能なフィードバックを確保するのに役立ちます。

In [ ]:
def check_match(reference_image_base64, suggested_image_base64):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": """ You will be given two images of two different items of clothing.
                            Your goal is to decide if the items in the images would work in an outfit together.
                            The first image is the reference item (the item that the user is trying to match with another item).
                            You need to decide if the second item would work well with the reference item.
                            Your response must be a JSON output with the following fields: "answer", "reason".
                            The "answer" field must be either "yes" or "no", depending on whether you think the items would work well together.
                            The "reason" field must be a short explanation of your reasoning for your decision. Do not include the descriptions of the 2 images.
                            Do not include the ```json ``` tag in the output.
                           """,
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{reference_image_base64}",
                },
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{suggested_image_base64}",
                },
                }
            ],
            }
        ],
        max_tokens=300,
    )
    # Extract relevant features from the response
    features = response.choices[0].message.content
    return features

最後に、上記で特定したアイテムの中から、実際にその服装を引き立てるものを判断しましょう。

In [ ]:
# Select the unique paths for the generated images
paths = list(set(paths))

for path in paths:
    # Encode the test image to base64
    suggested_image = encode_image_to_base64(path)
    
    # Check if the items match
    match = json.loads(check_match(encoded_image, suggested_image))
    
    # Display the image and the analysis results
    if match["answer"] == 'yes':
        display(Image(filename=path))
        print("The items match!")
        print(match["reason"])

初期の候補アイテムリストがさらに絞り込まれ、コーディネートによく合うより厳選されたセレクションになっていることが確認できます。さらに、モデルは各アイテムがなぜ良いマッチと考えられるかの説明を提供し、意思決定プロセスに関する貴重な洞察を提供しています。

### 結論

このJupyter Notebookでは、GPT-4o miniおよびその他の機械学習技術をファッション分野に応用することを探求しました。衣類アイテムの画像を分析し、関連する特徴を抽出し、この情報を使用して元の服装を補完するマッチングアイテムを見つける方法を実演しました。ガードレールと自己修正メカニズムの実装を通じて、モデルの提案を改良し、正確で文脈に関連性があることを確保しました。

このアプローチは、現実世界でいくつかの実用的な用途があります：

1. **パーソナライズされたショッピングアシスタント**: 小売業者はこの技術を使用して、顧客にパーソナライズされた服装の推奨を提供し、ショッピング体験を向上させ、顧客満足度を高めることができます。
2. **バーチャルワードローブアプリケーション**: ユーザーは自分の衣類アイテムの画像をアップロードしてバーチャルワードローブを作成し、既存のアイテムにマッチする新しいアイテムの提案を受けることができます。
3. **ファッションデザインとスタイリング**: ファッションデザイナーやスタイリストは、このツールを使用してさまざまな組み合わせやスタイルを実験し、創作プロセスを合理化できます。

ただし、考慮すべき点の一つは**コスト**です。LLMと画像分析モデルの使用は、特に広範囲に使用される場合、コストが発生する可能性があります。これらの技術を実装する際の費用対効果を考慮することが重要です。`gpt-4o-mini`は1000トークンあたり`$0.01`で価格設定されています。これは256px x 256pxの画像1枚あたり`$0.00255`になります。

全体として、このノートブックはファッションとAIの交差点におけるさらなる探求と開発の基盤として機能し、よりパーソナライズされたインテリジェントなファッション推奨システムへの扉を開きます。